In [ ]:
import pandas as pd

#load the data
with open('../data/true/false_geopardie.txt') as f:
    lines = f.read()
    
lines = lines.split('\n\n')
lines = lines[0:40]

In [ ]:
#parse data
df = pd.DataFrame([x.split('\n') for x in lines], columns=['Q','R','T','F1','F2'])
df['Q'] = df['Q'].str[4:].str[:-2]
df['R'] = df['R'].str[3:]
df['T'] = df['T'].str[3:]
df['F1'] = df['F1'].str[4:]
df['F2'] = df['F2'].str[4:]

In [ ]:
#save data
df.to_csv('../data/complex/geopardie.csv', index=False)

In [ ]:
#load model to generate activations
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")
%load_ext autoreload
%autoreload 2

import os
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {} '.format(device))

In [ ]:
from tqdm import tqdm

nb_neurons_per_layer = model.model.layers[5].self_attn.k_proj.in_features
nb_layer = model.model.layers.__len__()

activations = {}
#list the layer you want to extract the activations from
#layers_num = [i for i in range(0,nb_layer)]
layers_num = [0,7,10,12,15,22]
hooks = []
model_name = "Qwen1.5-0.5B"

class StopForwarding(Exception):
    pass

#hook to capture the output of the layers
def capture_output(module, input, output, layer_num):
    output = output[0].detach()
    activations[f'layer_{layer_num}_output'] = output[:,-1,:]
    if layer_num == layers_num[-1]:
        raise StopForwarding

#register the hook to the layers
for layer_num in layers_num:
    hooks.append(model.model.layers[layer_num].register_forward_hook(lambda module, input, output, layer_num=layer_num: capture_output(module, input, output, layer_num)))

raw_data = pd.read_csv(f"../data/complex/geopardie.csv")

#process the data
pbar = tqdm(total=raw_data.shape[0], desc=f"Processing lines of geopardie", unit="line")
pbar.update(0)
for i in range(0,raw_data.shape[0]) :
    #for each statement
    for col in ['T','F1','F2']:
        try : 
            #get the statement and the label
            statement = raw_data.iloc[i][col]
            label = col[0]
            #tokenize the statement and get the mask
            input_token = tokenizer.encode_plus(statement, return_tensors="pt")
            input_ids = input_token["input_ids"].to(device)
            mask = input_token["attention_mask"].to(device)
            #forward the statement through the model
            with torch.no_grad():
                output = model(input_ids=input_ids, attention_mask=mask)
                
        except StopForwarding:
            #save the activations
            for layer_num in layers_num:
                layer_output = activations[f'layer_{layer_num}_output']
                layer_output = layer_output.cpu().numpy()
                start_neuron = nb_neurons_per_layer * layer_num +1
                end_neuron = start_neuron + nb_neurons_per_layer
                layer_output = pd.DataFrame(layer_output, columns=[f'neuron_{i}' for i in range(start_neuron, end_neuron)])
                layer_output["label"] = label
                header = i == 0
                layer_output.to_csv(f"../activations/complex/geopardie/layer_{layer_num}_output.csv", mode='a', index=False, header=header)
    
    pbar.update(1)
pbar.close()
#remove the hooks to avoid memory leaks
for hook in hooks:
    hook.remove()

In [46]:
num=22
#load csv 
geopardie = pd.read_csv(f'../activations/complex/geopardie/layer_{num}_output.csv')
geopardie.shape

(122, 1025)

In [47]:
#delete row 1 and 3 
geopardie = geopardie.drop([1,3])

In [48]:
#save the data
geopardie.to_csv(f'../activations/complex/geopardie/layer_{num}_output.csv', index=False)